In [22]:
import pandas as pd 
import numpy as np
from datetime import date, datetime, timedelta
import lxml.html
import json
import requests 
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import json
import re
#---
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
url='https://www.segundamano.mx/anuncios/mexico/venta-inmuebles/oficinas-locales'
headers={'authority': 'www.segundamano.mx',
'path': '/anuncios/mexico/venta-inmuebles/oficinas-locales',
'upgrade-insecure-requests': '1',
'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36 Edg/104.0.1293.70'}
session=requests.Session()
session.verify=False 
r = session.get( url.replace(' ',''), headers=headers )
session.close()
root = lxml.html.fromstring(r.text)
Tabla=pd.DataFrame([[i['item']['name'], i['item']['url']] for i in json.loads(root.xpath("////*/script[@type = 'application/ld+json']/text()")[0])['itemListElement']],columns=['nombre','url'])

IndexError: list index out of range

In [21]:
r.text

'<!doctype html><html data-locale=es_MX lang=es-MX><head><meta charset=UTF-8><meta name=viewport content="width=device-width,initial-scale=1"><meta name=apple-itunes-app content="app-id=934844916"><link rel=manifest href="/manifest-android.json"><link rel="shortcut icon" href="/anuncios/static/favicon.ico" type="image/x-icon"><link rel=dns-prefetch href="https://www.google-analytics.com/"><link rel=preconnect href="https://www.google-analytics.com/" crossorigin><link rel=dns-prefetch href="https://ib.adnxs.com"><link rel=preconnect href="https://ib.adnxs.com" crossorigin><meta name=robots content="index, follow"><title>Venta de Oficinas/locales en México | Segundamano.mx</title><meta name=description content="5724 anuncios de venta inmuebles   🏘️ en venta de particulares y profesionales de segundamano y ocasión en México ✔️. Entra, revisa y encuentra 🔎 en Segundamano.mx."><meta property=og:type content=product.group><meta property=og:description content="5724 anuncios de venta inmueble

In [18]:
root.xpath("//script")

[]

In [2]:
def check_inside(c_json,nombre,dev):
  if nombre in c_json:
        return(c_json[nombre] )
  else: 
        return(dev)

def check_inside_cadena(c_json,nombre,restante,dev):
  if check_inside(c_json,nombre,0)!=0:
        return(eval('c_json'+restante) )
  else: 
        return(dev)
data2=[]
for url in Tabla.url:
  requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
  headers={ 
  'authority': 'www.segundamano.mx',
  'path': '/anuncios/mexico/venta-inmuebles/oficinas-locales',
  'upgrade-insecure-requests': '1',
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36 Edg/104.0.1293.70'
  }
  session=requests.Session()
  session.verify=False 
  r = session.get( url.replace(' ',''), headers=headers )
  session.close()
  root = lxml.html.fromstring(r.text)
  r1=root.xpath("//script[contains(.,'var AD =')]")[0].text
  #values = re.findall(r'var .*?=\s*(.*?);', r1, re.DOTALL | re.MULTILINE)
  #data=values[0]
  step_1 = r1.split('var AD =')
  step_2 = " ".join(step_1[1].split())
  step_3 = step_2.split('var SMMX_AD_VIEW ')
  step_4= step_3[0].replace('}, {','}xxx{').split('xxx')
  c_json=json.loads(step_4[0])
  precio=check_inside(c_json['prices'][0],'price_value','')
  image='https://images.segundamano.mx/api/v1/smmx/images/' + check_inside(c_json['thumbnail'],'path',0) + '?rule=webp_web_gallery_2x'
  estado=check_inside(c_json['locations'][0],'label','')
  municipio=check_inside(c_json['locations'][0]['locations'][0],'label','')
  sector=check_inside(c_json['locations'][0]['locations'][0]['locations'][0],'label','')
  detalle=check_inside(c_json,'body','')
  fec_ini=check_inside(c_json['list_time'],'label','')
  tipo=check_inside(c_json['type'],'label','')
  bath=check_inside_cadena(c_json['ad_details'],'bathrooms',"['bathrooms']['single']['label']",'')
  mail=check_inside_cadena(c_json['ad_details'],'contact_email',"['contact_email']['single']['label']",'')
  tipo2=check_inside(c_json['ad_details']['estate_type']['single'],'label','')
  estacionamiento=check_inside_cadena(c_json['ad_details'],'parking_lots',"['parking_lots']['single']['label']",'')
  area=check_inside_cadena(c_json['ad_details'],'size',"['size']['single']['label']",'')
  requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
  url='https://webapi.segundamano.mx/nga/api/v1/public/klfst/' + c_json['list_id'].split('/')[-1] + '/phone?lang=es'
  headers={ 
  'authority': 'webapi.segundamano.mx',
  'path': '/nga/api/v1/public/klfst/' + c_json['list_id'].split('/')[-1] + '/phone?lang=es',
  'upgrade-insecure-requests': '1',
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36 Edg/104.0.1293.70'
  }
  session=requests.Session()
  session.verify=False 
  r = session.get( url.replace(' ',''), headers=headers )
  session.close()
  root = lxml.html.fromstring(r.text)
  if check_inside(json.loads(r.text),'phones',0)!=0:
    telefono=check_inside(json.loads(r.text)['phones'][0],'value',0)
  else :
    telefono=''
  data2=data2+[[precio,image,estado,municipio,sector,detalle,fec_ini,tipo,bath,mail,tipo2,estacionamiento,area,telefono]]
segundamano=Tabla.join(pd.DataFrame(data2, columns=['precio','image','estado','municipio','sector','detalle','fec_ini','tipo','bath','mail','tipo2','estacionamiento','area','telefono']))

In [6]:
root.xpath("////*/script[@type = 'application/ld+json']/text()")

[]

In [ ]:
segundamano.to_csv('segundamano-ejemplo.csv')

In [3]:
segundamano

,nombre,url,precio,image,estado,municipio,sector,detalle,fec_ini,tipo,bath,mail,tipo2,estacionamiento,area,telefono
0,Se vende Consultorio u Oficina en Glorieta de,https://www.segundamano.mx/anuncios/ciudad-de-...,5850000,https://images.segundamano.mx/api/v1/smmx/imag...,Ciudad de México,Benito Juárez,Letrán Valle,Se vende Consultorio u Oficina en Glorieta de ...,26 Aug 22 11:56 CDT,Vendo,2,contacto@remax-futuro.com.mx,Oficinas/locales,1,75 m²,5589393740
1,Local en Venta en El Pueblito,https://www.segundamano.mx/anuncios/queretaro/...,1300000,https://images.segundamano.mx/api/v1/smmx/imag...,Querétaro,Corregidora,El Pueblito,Nocnok ID: MX22-NI7739. LOCAL EN VENTA EN PLAZ...,26 Aug 22 11:58 CDT,Vendo,,,Oficinas/locales,,33 m²,4421808209
2,Local en Venta en San Isidro,https://www.segundamano.mx/anuncios/queretaro/...,1850000,https://images.segundamano.mx/api/v1/smmx/imag...,Querétaro,Querétaro,Otra,Colonia: San Isidro\n\nNocnok ID: MX22-MN2468....,26 Aug 22 11:57 CDT,Vendo,,,Oficinas/locales,,51 m²,4428242792
3,LOCAL COMERCIAL EN VENTA EN PLAZA VIA,https://www.segundamano.mx/anuncios/sinaloa/cu...,2310000,https://images.segundamano.mx/api/v1/smmx/imag...,Sinaloa,Culiacán,Humaya,"Local comercial en venta ubicado en plaza vía,...",26 Aug 22 11:55 CDT,Vendo,,,Oficinas/locales,,46 m²,6677210021
4,Local en Venta en La Loma,https://www.segundamano.mx/anuncios/queretaro/...,2050000,https://images.segundamano.mx/api/v1/smmx/imag...,Querétaro,Querétaro,La Loma,Nocnok ID: MX19-HF2398. VENTA DE LOCAL COMERCI...,26 Aug 22 11:54 CDT,Vendo,,,Oficinas/locales,,88 m²,4423886107
5,"Consultorio Nuevo en Venta en Torre Medica, Tu...",https://www.segundamano.mx/anuncios/chiapas/tu...,1633771,https://images.segundamano.mx/api/v1/smmx/imag...,Chiapas,Tuxtla Gutiérrez,Santa Elena,Ref: NEX-118359\n\nConsultorio Nuevo en Venta ...,26 Aug 22 11:53 CDT,Vendo,1,ncc@neximo.mx,Oficinas/locales,1,38 m²,5570055984
6,"Consultorio Nuevo en Venta en Torre Medica, Tu...",https://www.segundamano.mx/anuncios/chiapas/tu...,2277506,https://images.segundamano.mx/api/v1/smmx/imag...,Chiapas,Tuxtla Gutiérrez,Santa Elena,Ref: NEX-113752\n\nConsultorio u Oficina en ve...,26 Aug 22 11:53 CDT,Vendo,1,ncc@neximo.mx,Oficinas/locales,1,50 m²,5570055984
7,"Consultorio Nuevo en Venta en Torre Medica, Tu...",https://www.segundamano.mx/anuncios/chiapas/tu...,2852813,https://images.segundamano.mx/api/v1/smmx/imag...,Chiapas,Tuxtla Gutiérrez,Santa Elena,Ref: NEX-113754\n\nConsultorio u Oficina en ve...,26 Aug 22 11:53 CDT,Vendo,1,ncc@neximo.mx,Oficinas/locales,1,57 m²,5570055984
8,"Consultorio Nuevo en Venta en Torre Medica, Tu...",https://www.segundamano.mx/anuncios/chiapas/tu...,1236465,https://images.segundamano.mx/api/v1/smmx/imag...,Chiapas,Tuxtla Gutiérrez,Santa Elena,Ref: NEX-113748\n\nConsultorio u Oficina en ve...,26 Aug 22 11:53 CDT,Vendo,1,ncc@neximo.mx,Oficinas/locales,1,33 m²,5570055984
9,"Consultorio Nuevo en Venta en Torre Medica, Tu...",https://www.segundamano.mx/anuncios/chiapas/tu...,1498872,https://images.segundamano.mx/api/v1/smmx/imag...,Chiapas,Tuxtla Gutiérrez,Santa Elena,Ref: NEX-113749\n\nConsultorio u Oficina en ve...,26 Aug 22 11:53 CDT,Vendo,1,ncc@neximo.mx,Oficinas/locales,1,38 m²,5570055984


In [ ]:
step_3 

['{"list_id":"/public/ads/935351347","list_price":{"currency":"MXN","label":"$ 14,000,000","price_value":14000000},"prices":[{"currency":"MXN","label":"$ 14,000,000","price_value":14000000}],"share_link":"https://www.segundamano.mx/anuncios/chiapas/tuxtla-gutierrez/venta-inmuebles/casa-en-venta-935351347","subject":"CASA EN VENTA","thumbnail":{"path":"06/0690162942.jpg","base_url":"http://img.segundamano.mx"},"images":[{"path":"06/0690162942.jpg","base_url":"http://img.segundamano.mx"},{"path":"06/0698686261.jpg","base_url":"http://img.segundamano.mx"},{"path":"06/0615732899.jpg","base_url":"http://img.segundamano.mx"},{"path":"06/0607209580.jpg","base_url":"http://img.segundamano.mx"}],"user":{"account":{"name":"Century 21 Optima","since":"2017-08-29 17:16:45.767649","user_uuid":"28d68236-ca9d-4a01-8b93-e427a046af5f"}},"locations":[{"code":"9","label":"Chiapas","locations":[{"code":"211","label":"Tuxtla Gutiérrez","locations":[{"code":"140000","label":"Otra","locations":null}]}]}],"sh

In [ ]:
'{"list_id":"/public/ads/935351347","list_price":{"currency":"MXN","label":"$ 14,000,000","price_value":14000000},"prices":[{"currency":"MXN","label":"$ 14,000,000","price_value":14000000}],"share_link":"https://www.segundamano.mx/anuncios/chiapas/tuxtla-gutierrez/venta-inmuebles/casa-en-venta-935351347","subject":"CASA EN VENTA","thumbnail":{"path":"06/0690162942.jpg","base_url":"http://img.segundamano.mx"},"images":[{"path":"06/0690162942.jpg","base_url":"http://img.segundamano.mx"},{"path":"06/0698686261.jpg","base_url":"http://img.segundamano.mx"},{"path":"06/0615732899.jpg","base_url":"http://img.segundamano.mx"},{"path":"06/0607209580.jpg","base_url":"http://img.segundamano.mx"}],"user":{"account":{"name":"Century 21 Optima","since":"2017-08-29 17:16:45.767649","user_uuid":"28d68236-ca9d-4a01-8b93-e427a046af5f"}},"locations":[{"code":"9","label":"Chiapas","locations":[{"code":"211","label":"Tuxtla Gutiérrez","locations":[{"code":"140000","label":"Otra","locations":null}]}]}],"shop":{"id":3381},"body":"Colonia: Las Granjas Boule',


('{"list_id":"/public/ads/935351347","list_price":{"currency":"MXN","label":"$ 14,000,000","price_value":14000000},"prices":[{"currency":"MXN","label":"$ 14,000,000","price_value":14000000}],"share_link":"https://www.segundamano.mx/anuncios/chiapas/tuxtla-gutierrez/venta-inmuebles/casa-en-venta-935351347","subject":"CASA EN VENTA","thumbnail":{"path":"06/0690162942.jpg","base_url":"http://img.segundamano.mx"},"images":[{"path":"06/0690162942.jpg","base_url":"http://img.segundamano.mx"},{"path":"06/0698686261.jpg","base_url":"http://img.segundamano.mx"},{"path":"06/0615732899.jpg","base_url":"http://img.segundamano.mx"},{"path":"06/0607209580.jpg","base_url":"http://img.segundamano.mx"}],"user":{"account":{"name":"Century 21 Optima","since":"2017-08-29 17:16:45.767649","user_uuid":"28d68236-ca9d-4a01-8b93-e427a046af5f"}},"locations":[{"code":"9","label":"Chiapas","locations":[{"code":"211","label":"Tuxtla Gutiérrez","locations":[{"code":"140000","label":"Otra","locations":null}]}]}],"sh